In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 80% !important; }</style>"))

In [2]:
##### 대신증권 연결 확인
import win32com.client

instCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
print(instCpCybos.IsConnect)

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from sklearn.preprocessing import minmax_scale
from statsmodels.tsa.stattools import coint
import stock_order
today = datetime.today().strftime("%Y%m%d") 
start = (datetime.today() - timedelta(5)).strftime("%Y%m%d") 
print(start, today)

1
20201219 20201224


# Tools

In [3]:
class Order():
    
    def sell(code,num):
    # 주식 매수 주문
        # 주문 초기화
        objTrade =  win32com.client.Dispatch("CpTrade.CpTdUtil")
        initCheck = objTrade.TradeInit(0)
        if (initCheck != 0):
            print("주문 초기화 실패")
            exit()
            
        acc = objTrade.AccountNumber[0] #계좌번호
        accFlag = objTrade.GoodsList(acc, 1)  # 주식상품 구분
        print(acc, accFlag[0])
        objStockOrder = win32com.client.Dispatch("CpTrade.CpTd0311")
        objStockOrder.SetInputValue(0, "1")   # 1: 매도
        objStockOrder.SetInputValue(1, acc )   #  계좌번호
        objStockOrder.SetInputValue(2, accFlag[0])   # 상품구분 - 주식 상품 중 첫번째
        objStockOrder.SetInputValue(3, code)   # 종목코드 
        objStockOrder.SetInputValue(4, num)   # 매수수량 
#         objStockOrder.SetInputValue(5, price)   # 주문단가  
        objStockOrder.SetInputValue(7, "0")   # 주문 조건 구분 코드, 0: 기본 1: IOC 2:FOK
        objStockOrder.SetInputValue(8, "03")   # 주문호가 구분코드 - 01: 보통 , 03:시장가

        # 매수 주문 요청
        objStockOrder.BlockRequest()

        rqStatus = objStockOrder.GetDibStatus()
        rqRet = objStockOrder.GetDibMsg1()
        print("통신상태", rqStatus, rqRet)
        if rqStatus != 0:
            exit()
        
    def buy(code,num):
        # 주식 매수 주문
        # 주문 초기화
        objTrade =  win32com.client.Dispatch("CpTrade.CpTdUtil")
        initCheck = objTrade.TradeInit(0)
        if (initCheck != 0):
            print("주문 초기화 실패")
            exit()        
        
        acc = objTrade.AccountNumber[0] #계좌번호
        accFlag = objTrade.GoodsList(acc, 1)  # 주식상품 구분
        print(acc, accFlag[0])
        objStockOrder = win32com.client.Dispatch("CpTrade.CpTd0311")
        objStockOrder.SetInputValue(0, "2")   # 2: 매수
        objStockOrder.SetInputValue(1, acc )   #  계좌번호
        objStockOrder.SetInputValue(2, accFlag[0])   # 상품구분 - 주식 상품 중 첫번째
        objStockOrder.SetInputValue(3, code)   # 종목코드 
        objStockOrder.SetInputValue(4, num)   # 매수수량 
#         objStockOrder.SetInputValue(5, price)   # 주문단가  
        objStockOrder.SetInputValue(7, "0")   # 주문 조건 구분 코드, 0: 기본 1: IOC 2:FOK
        objStockOrder.SetInputValue(8, "03")   # 주문호가 구분코드 - 01: 보통 , 03:시장가

        # 매수 주문 요청
        objStockOrder.BlockRequest()

        rqStatus = objStockOrder.GetDibStatus()
        rqRet = objStockOrder.GetDibMsg1()
        print("통신상태", rqStatus, rqRet)
        if rqStatus != 0:
            exit()
        
    def get_min(code,time,day):  # 종목, 기간, 오늘, 시점, 분, 시간간격
        start = (datetime.today() - timedelta(day)).strftime("%Y%m%d") 
        instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
        instStockChart.SetInputValue(0, code )
        instStockChart.SetInputValue(1, ord('1'))
        instStockChart.SetInputValue(2, today)
        instStockChart.SetInputValue(3, start)
        # instStockChart.SetInputValue(4, 1000)
        instStockChart.SetInputValue(5, (0,1,5))
        instStockChart.SetInputValue(6, ord(time))  # 'm' : 분, 'T' : 틱
        instStockChart.SetInputValue(7, 1)      # 데이터 주기
        instStockChart.SetInputValue(9, ord('1'))
        instStockChart.SetInputValue(10, 3)

        instStockChart.BlockRequest()

        numData = instStockChart.GetHeaderValue(3)
        numField = instStockChart.GetHeaderValue(1) 

        temp = {}
        for i in range(numData):
            temp[str(instStockChart.GetDataValue(0, i)) +'.'+ str(instStockChart.GetDataValue(1, i))] = [instStockChart.GetDataValue(2, i)]
        temp = pd.DataFrame(temp).transpose()
        temp.index.names = ['time']
        return temp
    
    def get_data(time,day):
        kospi = Order.get_min('A005930',time,day)
        kosdaq = Order.get_min('A069500',time,day)
        temp =  pd.merge(left = kospi , right =kosdaq, how = "inner", on = "time")
        temp.columns = ['kospi','kosdaq']
        
        return temp['kospi'], temp['kosdaq']
    
    def present_price(code):
        
        import win32com.client

        # 연결 여부 체크
        objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
        bConnect = objCpCybos.IsConnect
        if (bConnect == 0):
            print("PLUS가 정상적으로 연결되지 않음. ")
            exit()

        # 현재가 객체 구하기
        objStockMst = win32com.client.Dispatch("DsCbo1.StockMst")
        objStockMst.SetInputValue(0, code)   
        objStockMst.BlockRequest()

        # 현재가 통신 및 통신 에러 처리 
        rqStatus = objStockMst.GetDibStatus()
        rqRet = objStockMst.GetDibMsg1()
        print("통신상태", rqStatus, rqRet)
        if rqStatus != 0:
            exit()

        # 현재가 정보 조회    
        return objStockMst.GetHeaderValue(11) # 종가
    

# 알고리즘

In [4]:
#Trade using a simple strategy
def trade(S1, S2, window1, window2):
    
    #If window length is 0, algorithm doesn't make sense, so exit
    if (window1 == 0) or (window2 == 0):
        return 0
    
    #Compute rolling mean and rolling standard deviation
#     ratios = S1/S2
    spread = S1 - 2.5* S2
    ma1 = spread.rolling(window=window1,center=False).mean()
    ma2 = spread.rolling(window=window2,center=False).mean()
    std = spread.rolling(window=window2, center=False).std()
    zscore = (ma1 - ma2)/std

    kospi_x2 = 'A122630' ; kospi_x2inv = 'A252670' ; kosdaq_x2= 'A233740' ; kosdaq_inv = 'A251340'
    
    #Sell short if the z-score is > 1
    if zscore[-1] > 1:   # sell 코스닥  buy 코스닥

        Order.buy(kospi_x2inv,2) ;#숏포지션
        Order.buy(kosdaq_x2,5) ; #롱포지션

        total_eval, total_profit,name, amount, price = stock_order.get_info()

        #print('[SELL_INFO]',' 코스피 매도금액 :',price[0]* (1-0.004936563),'/ 코스닥 매수금액 :',price[1]*2* (1.004936563))
        print('[SELL]' ,'총평가금액: ',round(total_eval,2),'원','총평가손익 :',total_profit, '코스피 :',-amount[3],'주  /  코스닥:',amount[1],'주' )
    
    # Buy long if the z-score is < 1
    elif zscore[-1] < -1: # buy 코스닥 sell 코스닥
             
        Order.buy(kospi_x2,2)     #롱포지션
        Order.buy(kosdaq_inv,10)  #숏포지션

        total_eval, total_profit,name, amount, price = stock_order.get_info()

        #print('[BUY_INFO]',' 코스피 매수금액 :',price[0]* (1.004936563),'/ 코스닥 매도금액 :',price[1] * 2 * (1-0.004936563))
        print('[BUY]' ,'총평가금액: ',round(total_eval,2),'원','총평가손익 :',total_profit,'코스피 :',amount[0],'주  /  코스닥 :',-amount[2],'주' )

    # Clear positions if the z-score between -.5 and .5        
    # -0.5~0.5 사이인 경우 수익일 경우 이익 실현    
    elif abs(zscore[-1]) < 0.5 :                         
        
        total_eval, total_profit,name, amount, price = stock_order.get_info()

        if total_profit > 0 :                
            if amount[0] > 0 : # kospi에 숏포지션 한 경우                   

                Order.sell( kospi_x2inv, amount[3], price[3] )
                Order.sell( kosdaq_x2, amount[1], price[1])

#                 print('[EXIT]' ,'총평가금액: ',round(total_eval,2),'원', '코스피 :',amount[0],'주  /  코스닥:',amount[1],'주' ) 
                print('[EXIT]' ,'총평가금액: ',round(total_eval,2),'원')
            elif amount[0] > 0:# kospi에 롱포지션 한 경우 

                Order.sell( kospi_x2, amount[0],price[0])
                Order.sell( kosdaq_inv, amount[2] ,price[2] )

#                 print('[EXIT]' ,'총평가금액: ',round(total_eval,2),'원', '코스피 :',amount[0],'주  /  코스닥:',amount[1],'주' ) 
                print('[EXIT]' ,'총평가금액: ',round(total_eval,2),'원')
    
    else:
        pass
    now = datetime.now()
    total_eval, total_profit, name,amount, price = stock_order.get_info()
    print('')
    print('[',now,']')
    print('-------------------------------현재상태--------------------------------------')
    print('총평가금액 :',total_eval,'총평가손익 :',total_profit)
    for i in range(1,len(price)):
        print(name[i], ',', price[i],',',amount[i])
    print('-----------------------------------------------------------------------------')
    print('')              
    

# 트레이딩

In [ ]:
import win32com.client
import stock_order 
from datetime import datetime,timedelta
import time
today = datetime.today().strftime("%Y%m%d") 
start = (datetime.today() - timedelta(5)).strftime("%Y%m%d") 
now = datetime.now()

# 연결 여부 체크
objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
bConnect = objCpCybos.IsConnect
if (bConnect == 0):
    print("PLUS가 정상적으로 연결되지 않음. ")
    exit()

# Start with no money and no positions 


print('-------------------------------거래시작------------------------------------')
while now.strftime('%H%M%S') != 151501:
    now = datetime.now()
    S1,S2 = Order.get_data(1,14)   # 분봉, 일간격
    profit = trade(S1, S2, 1, 20) # windowsize 1,2
#     print(now,' ------------- 현재 총 평가금액 : ',profit,'--------------------------')
    time.sleep(10)
#코스피레버리지, 코스닥레버리지, 코스닥인버스, 코스피인버스

-------------------------------거래시작------------------------------------


In [ ]:
now = datetime.now()
total_eval, total_profit, name,amount, price = stock_order.get_info()

print('')
print('[',now,']')
print('-------------------------------현재상태--------------------------------------')
print('총평가금액 :',total_eval,'총평가손익 :',total_profit)
for i in range(1,len(price)):
    print(name[i], ',', price[i],',',amount[i])
print('-----------------------------------------------------------------------------')
print('')